In [2]:
library("spatialLIBD")

Loading required package: SpatialExperiment

Loading required package: SingleCellExperiment

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: 'MatrixGenerics'


The following objects are masked from 'package:matrixStats':

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, 

In [3]:
## Download the small example sce data
#sce <- readRDS("../data/spatialDLPFC_Visium/spe_filtered_final_with_clusters_and_deconvolution_results.rds")
load(file = "../data/spatialDLPFC_Visium/sce_sub_for_vignette.Rdata")
#load(file = "../data/spatialDLPFC_Visium/Human_DLPFC_Visium_processedData_sce_scran_spatialLIBD.Rdata")

In [5]:
sce_to_spe <- function(sce = fetch_data("sce"), imageData = NULL) {

    # Load assays
    assays_visium <- SummarizedExperiment::assays(sce)

    # Load rowData
    rowData_visium <- SummarizedExperiment::rowData(sce)

    ## Re-case tissue
    if (is.factor(sce$tissue)) sce$tissue <- sce$tissue == "1"

    # Load colData
    cols_to_drop <-
        c(
            "tissue",
            "row",
            "col",
            "imagerow",
            "imagecol"
        )
    colData_visium <-
        SummarizedExperiment::colData(sce)[, !colnames(SummarizedExperiment::colData(sce)) %in% c(cols_to_drop, "height", "width", "barcode"), drop = FALSE]

    names(colData_visium)[names(colData_visium) == "sample_name"] <- "sample_id"

    # Load spatialCoords
    spatialCoords_visium <-
        SummarizedExperiment::colData(sce)[, colnames(SummarizedExperiment::colData(sce)) %in% cols_to_drop, drop = FALSE]
    names(spatialCoords_visium) <-
        c(
            "in_tissue",
            "array_row",
            "array_col",
            "pxl_row_in_fullres",
            "pxl_col_in_fullres"
        )

    ## Use the official sample id name
    colnames(colData_visium)[colnames(colData_visium) == "sample_name"] <- "sample_id"
    colData_visium$sample_id <- as.character(colData_visium$sample_id)

    # Load reducedDim
    reducedDimNames_visium <-
        SingleCellExperiment::reducedDims(sce)

    # Load images from the web for our data
    if (is.null(imageData)) {
        sample_id <- unique(colData_visium$sample_id)

        url_images <-
            paste0(
                "https://spatial-dlpfc.s3.us-east-2.amazonaws.com/images/",
                sample_id,
                "_tissue_lowres_image.png"
            )

        # Load scaleFactors
        url_scaleFactors <- paste0(
            "https://raw.githubusercontent.com/LieberInstitute/",
            "HumanPilot/master/10X/",
            sample_id,
            "/scalefactors_json.json"
        )
        names(url_scaleFactors) <- sample_id
        scaleFactors_visium <-
            lapply(url_scaleFactors, jsonlite::read_json)

        ## Create a list of images

        spatial_img_list <- mapply(function(url) {
            SpatialExperiment::SpatialImage(
                url
            )
        }, url_images)


        img_dat <- DataFrame(
            sample_id = as.character(sample_id),
            image_id = rep("lowres", length(sample_id)),
            data = I(spatial_img_list),
            scaleFactor = vapply(
                scaleFactors_visium,
                "[[",
                numeric(1),
                "tissue_lowres_scalef",
                USE.NAMES = FALSE
            )
        )
        imageData <- img_dat

        ## Fix things we had done that are not default:
        # Scaling for lowres image: https://github.com/LieberInstitute/HumanPilot/blob/master/Analysis/Layer_Notebook.R#L118-L119
        spatialCoords_visium$pxl_col_in_fullres <- spatialCoords_visium$pxl_col_in_fullres / img_dat$scaleFactor[match(colData_visium$sample_id, img_dat$sample_id)]
        spatialCoords_visium$pxl_row_in_fullres <- spatialCoords_visium$pxl_row_in_fullres / img_dat$scaleFactor[match(colData_visium$sample_id, img_dat$sample_id)]
        ## Names of the columns is flipped at https://github.com/LieberInstitute/HumanPilot/blob/master/Analysis/Layer_Notebook.R#L116 compared to what
        ## SpatialExperiment does at https://github.com/drighelli/SpatialExperiment/blob/bf1b18b559ea2785d52db4e39a85f1d584aede45/R/read10xVisium.R#L170
        # tmp <- spatialCoords_visium$pxl_row_in_fullres
        # spatialCoords_visium$pxl_row_in_fullres <- spatialCoords_visium$pxl_col_in_fullres
        # spatialCoords_visium$pxl_col_in_fullres <- tmp
        ## The above is no longer necessary thanks to https://github.com/drighelli/SpatialExperiment/commit/6710fe8b0a7919191ecce989bb6831647385ef5f
    }

    # ## Create object manually
    # spe <- new("SpatialExperiment", SingleCellExperiment::SingleCellExperiment(
    #     rowData = rowData_visium,
    #     colData = colData_visium,
    #     assays = assays_visium,
    #     reducedDims = reducedDimNames_visium
    # ))
    #
    # ## Add missing spatial info
    # colData(spe) <- spatialCoords_visium
    # SpatialExperiment::spatialCoordsNames(spe) <- c("pxl_col_in_fullres", "pxl_row_in_fullres")
    # SpatialExperiment::imgData(spe) <- imageData
    #

    ## This works now in SpatialExperiment version 1.1.701, so we no longer
    ## need the manual code from above

    ## The following code ultimately fails due to the current lack of support
    ## for multiple `sample_id`s, as in
    ## https://github.com/drighelli/SpatialExperiment/blob/a9e54fbd5af7fe676f8a5b29e4cfe113402070d4/R/SpatialExperiment.R#L143-L144
    ## or in
    ## https://github.com/drighelli/SpatialExperiment/blob/a9e54fbd5af7fe676f8a5b29e4cfe113402070d4/R/SpatialExperiment.R#L164

    spe <- SpatialExperiment::SpatialExperiment(
        rowData = rowData_visium,
        colData = cbind(colData_visium, spatialCoords_visium),
        assays = assays_visium,
        reducedDims = reducedDimNames_visium,
        sample_id = NULL,
        spatialCoordsNames = c("pxl_col_in_fullres", "pxl_row_in_fullres"),
        scaleFactors = img_dat$scaleFactor,
        imgData = img_dat,
        imageSources = url_images,
        loadImage = FALSE
    )
    return(spe)
}

In [6]:
spe = sce_to_spe(sce = sce_sub)

In [7]:
spe

class: SpatialExperiment 
dim: 33538 47681 
metadata(0):
assays(2): counts logcounts
rownames(33538): ENSG00000243485 ENSG00000237613 ... ENSG00000277475
  ENSG00000268674
rowData names(9): source type ... gene_search is_top_hvg
colnames(47681): AAACAACGAATAGTTC-1 AAACAAGTATCTCCCA-1 ...
  TTGTTTCCATACAACT-1 TTGTTTGTGTAAATTC-1
colData names(68): sample_id Cluster ... array_row array_col
reducedDimNames(6): PCA TSNE_perplexity50 ... TSNE_perplexity80
  UMAP_neighbors15
mainExpName: NULL
altExpNames(0):
spatialCoords names(2) : pxl_col_in_fullres pxl_row_in_fullres
imgData names(4): sample_id image_id data scaleFactor